In [1]:
import string
import hashlib
import random
from pwn import *

DEBUG = False

def solve_pow(prefix):
    POW_CHARSET = string.ascii_letters + string.digits
    assert len(prefix) == 10

    print(f"solving pow {prefix}")

    ctr = 0
    while True:
        ctr += 1
        if ctr % 0x100000 == 0:
            print(".", end="", flush=True)
        guess = prefix + "".join(random.choices(POW_CHARSET, k=8))
        if hashlib.sha256(guess.encode('ascii')).hexdigest()[-6:] != "ffffff":
            continue
        print(" " + guess)
        return guess

r = None
charset = None
def connect_to_challenge(sub_challenge_name, use_charset):
    global r, charset
    if DEBUG:
        r = process(["python", "localrun.py"])
    else:
        r = remote("fastrology.chal.pwni.ng", 1337)
        r.recvuntil(b"with ")
        pow_chal = r.recvline().decode().strip().split()[0]
        r.sendline(solve_pow(pow_chal).encode())
    r.sendline(sub_challenge_name.encode())
    charset = use_charset

def moon_to_buckets(moon):
    return [ charset.index(i) for i in moon ]
def buckets_to_moon(buckets):
    return "".join([ charset[i] for i in buckets ])

chal_hash = None
def get_challenge():
    global chal_hash
    r.recvuntil(b"trial")
    r.recvline()
    chal_prefix = r.recvline().decode().strip()
    print("challenge:", chal_prefix[:20])
    chal_prefix = moon_to_buckets(chal_prefix)
    chal_hash = r.recvline().decode().strip()
    return chal_prefix

def answer_challenge(ans, do_send=False):
    ans = buckets_to_moon(ans)
    if hashlib.md5(ans.encode()).hexdigest() == chal_hash:
        print("answer:", ans[:20])
        r.sendline(ans.encode())
        return True
    else:
        return False
    
def claim_flag():
    global r
    r.recvuntil(b"congrats!\n")
    flag = r.recvline().decode()
    r = r.close()
    print(flag)
    return flag

In [2]:
import struct

def f64_to_u64(f):
    # f64 in [0, 1) to u64 XorShift128 state
    if f == 1.0:
        return 0xffffffffffffffff
    buf = struct.pack("d", f + 1)
    u52 = struct.unpack("<Q", buf)[0] & 0x000fffffffffffff
    u64 = u52 << 12
    return u64

def u64_to_f64(u):
    # u64 XorShift128 state to f64 in [0, 1)
    buf = struct.pack("<Q", (u >> 12) | 0x3FF0000000000000)
    f64 = struct.unpack("d", buf)[0] - 1
    return f64

In [3]:
class BV:
    def __init__(self, data):
        # data is 64-long list of 128 bit ints
        # data[0] is a 128 bit int indicating the mask of bits in s0s1
        #   which are xor-ed together to generate the 0th (LSB) bit of
        #   this state
        # Low 64 bits of s0s1 come from s0, high 64 bits come from s1
        assert(len(data) == 64)
        self.data = data
    
    def __xor__(self, other):
        return BV([i ^ j for i, j in zip(self.data, other.data)])

    def __lshift__(self, other):
        # After left shift the least significant bits of the state
        # should be empty
        return BV([0] * other + self.data[:-other])
    
    def __rshift__(self, other):
        # After right shift the most significant bits of the state
        # should be empty
        return BV(self.data[other:] + [0] * other)

    def coef(self, pos):
        # Converts 128 bit mask into list of ints
        coef = f"{self.data[pos]:0128b}"
        coef = [int(i) for i in coef[::-1]]
        return coef
    
    def eval_one(self, s0s1, pos):
        # From a known s0s1, evalute the bit at one position
        val = sum([i * j for i, j in zip(self.coef(pos), s0s1)]) % 2
        return val
    
    def eval_all(self, s0s1):
        # From a known s0s1, evalute the bit at all positions and get
        # the u64 output
        vals = [self.eval_one(s0s1, pos) for pos in range(64)]
        vals = "".join([str(i) for i in vals[::-1]])
        vals = int(vals, 2)
        return vals
    
def xs128p(state0, state1):
    s1 = state0
    s0 = state1
    
    s1 = s1 ^ (s1 << 23)
    s1 = s1 ^ (s1 >> 17)
    s1 = s1 ^ s0
    s1 = s1 ^ (s0 >> 26)

    output_state = state0
    state0 = state1
    state1 = s1

    return state0, state1, output_state

# Initial symbolic values of s0 and s1
BV.s0 = BV([1 << i for i in range(64)])
BV.s1 = BV([1 << i for i in range(64, 128)])

In [4]:
s0, s1 = BV.s0, BV.s1
s0, s1, output_state_0 = xs128p(s0, s1)
s0, s1, output_state_1 = xs128p(s0, s1)
s0, s1, output_state_2 = xs128p(s0, s1)

symbolic_state_2_bit_30 = f"{output_state_2.data[30]:0128b}"
print("The 30th bit of rng[2] depends on these bits of s0 and s1:")
print(f"s0 mask: {symbolic_state_2_bit_30[64:]}")
print(f"s1 mask: {symbolic_state_2_bit_30[:64]}")

The 30th bit of rng[2] depends on these bits of s0 and s1:
s0 mask: 0000000000000000100000000000000001000001000000000000000010000000
s1 mask: 0000000100000000000000000000000001000000000000000000000000000000


In [5]:
def schedule_sequence(seq):
    assert(len(seq) % 64 == 0) # ensure block aligned
    return [
        j
        for i in range(0, len(seq), 64)
        for j in seq[i:i+64][::-1]
    ]

n_steps = 64 * 70
s0, s1 = BV.s0, BV.s1
prng_states = []
for _ in range(n_steps):
    s0, s1, output_state = xs128p(s0, s1)
    prng_states.append(output_state)
prng_states = schedule_sequence(prng_states)

In [6]:
print(f"{f64_to_u64(0.0)  = :064b}")
print(f"{f64_to_u64(0.1)  = :064b}")
print(f"{f64_to_u64(0.2)  = :064b}")
print(f"{f64_to_u64(0.25) = :064b}")

f64_to_u64(0.0)  = 0000000000000000000000000000000000000000000000000000000000000000
f64_to_u64(0.1)  = 0001100110011001100110011001100110011001100110011010000000000000
f64_to_u64(0.2)  = 0011001100110011001100110011001100110011001100110011000000000000
f64_to_u64(0.25) = 0100000000000000000000000000000000000000000000000000000000000000


In [7]:
from sage.all_cmdline import Matrix, vector, GF

def tand(a, b):
    return "".join([
        i if i == j else "?"
        for i, j in zip(a, b)
    ])

high_bits_max_precision = 12
def fmt(x):
    return f"{x:012b}"

def generate_moon_to_fixed_msb(n_buckets):
    fractions = [
        f64_to_u64(i / n_buckets) >> (64 - high_bits_max_precision)
        for i in range(0, n_buckets+1)
    ]
    moon_to_fixed_msb = {}
    for idx, (i, j) in enumerate(zip(fractions, fractions[1:])):
        acc = fmt(i)
        for k in range(i, j + 1):
            acc = tand(acc, fmt(k))
        moon_to_fixed_msb[idx] = acc.rstrip("?")
    
    return moon_to_fixed_msb

In [8]:
moon_to_fixed_msb = {
    0: '00', 1: '01', 2: '10', 3: '11'
}

moon_to_fixed_msb_precomp = {
    k: [(63 - idx, int(fix_bit)) for idx, fix_bit in enumerate(v)]
    for k, v in moon_to_fixed_msb.items()
}

F = GF(2)

connect_to_challenge("waxing crescent", "☊☋☌☍")

for trial in range(50):
    print(f"trial {trial}")
    moons = get_challenge()
    ok = False

    for offset in range(64):
        system_mat = []
        system_vec = []
        math_random = iter(prng_states)

        for _ in range(offset):
            next(math_random)
        for moon in moons:
            moon_prng_state = next(math_random)
            for bit_pos, fix_value in moon_to_fixed_msb_precomp[moon]:
                system_mat.append(moon_prng_state.coef(bit_pos))
                system_vec.append(fix_value)
        
        try:
            s0s1 = Matrix(F, system_mat).solve_right(vector(F, system_vec))
            print(f"Found s0s1 with {offset = }")
        except ValueError:
            # no solution, offset is wrong
            continue

        soln = [
            int(u64_to_f64(prng_state.eval_all(s0s1)) * 4)
            for prng_state in prng_states[offset + 135: offset + 135 + 128]
        ]

        if answer_challenge(soln):
            ok = True
            break

    if not ok:
        raise ValueError("No solution found")
flag = claim_flag()

[x] Opening connection to fastrology.chal.pwni.ng on port 1337
[x] Opening connection to fastrology.chal.pwni.ng on port 1337: Trying 207.148.17.254
[+] Opening connection to fastrology.chal.pwni.ng on port 1337: Done
solving pow n73C9jsD0e
..................... n73C9jsD0eCheW6C05
trial 0
challenge: ☌☋☌☋☋☌☊☋☍☌☍☊☌☋☌☍☍☊☋☍
Found s0s1 with offset = 38
answer: ☋☊☊☊☌☌☋☍☌☌☊☋☌☋☍☌☍☊☌☋
trial 1
challenge: ☌☋☌☊☊☌☊☌☊☌☍☋☌☊☋☋☊☋☊☍
Found s0s1 with offset = 29
answer: ☋☊☌☍☍☋☌☌☍☍☋☊☊☌☋☋☌☍☍☋
trial 2
challenge: ☊☍☌☍☍☋☋☊☊☌☋☌☋☊☋☋☍☊☊☌
Found s0s1 with offset = 52
answer: ☍☍☍☋☋☍☌☌☊☊☍☍☋☊☊☋☌☊☋☊
trial 3
challenge: ☍☌☋☋☋☍☊☋☋☍☌☍☍☊☍☋☌☊☍☋
Found s0s1 with offset = 4
answer: ☊☍☊☍☊☋☍☋☍☋☌☌☌☌☌☌☍☍☋☋
trial 4
challenge: ☋☊☊☌☍☊☍☍☊☍☍☊☋☋☌☊☌☍☍☌
Found s0s1 with offset = 43
Found s0s1 with offset = 44
answer: ☊☊☊☌☊☌☊☌☊☊☋☊☊☌☊☍☋☋☌☋
trial 5
challenge: ☍☊☌☍☊☌☌☊☍☊☋☊☊☌☋☊☍☌☋☋
Found s0s1 with offset = 28
answer: ☌☊☍☋☋☋☋☊☍☍☍☍☌☌☋☌☍☍☌☍
trial 6
challenge: ☋☍☊☋☋☋☌☍☊☊☍☌☍☍☊☋☌☍☌☍
Found s0s1 with offset = 28
answer: ☌☊☌☍☊☍☌☊☍☍☍☌☋☊☊☊☋☍☍☊
trial 7
chal

In [9]:
moon_to_fixed_msb = generate_moon_to_fixed_msb(13)
    
print(f"{moon_to_fixed_msb = }")

moon_to_fixed_msb_precomp = {
    k: [(63 - idx, int(fix_bit)) for idx, fix_bit in enumerate(v)]
    for k, v in moon_to_fixed_msb.items()
}

F = GF(2)

connect_to_challenge("new moon", "♈♉♊♋♌♍♎♏♐♑♒♓⛎")

for trial in range(50):
    print(f"trial {trial}")
    moons = get_challenge()
    ok = False

    for offset in range(64):
        system_mat = []
        system_vec = []
        math_random = iter(prng_states)

        for _ in range(offset):
            next(math_random)
        for moon in moons:
            moon_prng_state = next(math_random)
            for bit_pos, fix_value in moon_to_fixed_msb_precomp[moon]:
                system_mat.append(moon_prng_state.coef(bit_pos))
                system_vec.append(fix_value)

        try:
            s0s1 = Matrix(F, system_mat).solve_right(vector(F, system_vec))
            print(f"Found s0s1 with {offset = }")
        except ValueError:
            # no solution, offset is wrong
            continue

        soln = [
            int(u64_to_f64(prng_state.eval_all(s0s1)) * 13)
            for prng_state in prng_states[offset + 192: offset + 192 + 128]
        ]

        if answer_challenge(soln):
            ok = True
            break

    if not ok:
        raise ValueError("No solution found")

flag = claim_flag()

moon_to_fixed_msb = {0: '000', 1: '00', 2: '001', 3: '0', 4: '01', 5: '011', 6: '', 7: '100', 8: '10', 9: '1', 10: '110', 11: '11', 12: '111'}
[x] Opening connection to fastrology.chal.pwni.ng on port 1337
[x] Opening connection to fastrology.chal.pwni.ng on port 1337: Trying 207.148.17.254
[+] Opening connection to fastrology.chal.pwni.ng on port 1337: Done
solving pow pm6fhd7ZOo
....... pm6fhd7ZOo6QQKi0PI
trial 0
challenge: ♎⛎⛎♓♈♑♑♈♐♐♍♓♈♎♍♍♒⛎♋♋
Found s0s1 with offset = 7
answer: ♊♊♏♈⛎♓♈♎♋♒⛎♈♏♌♌♉♎♍♓♉
trial 1
challenge: ♐♒♉♊♏♋♑⛎♓♏♏♓♎♉♏♏⛎♈♒♋
Found s0s1 with offset = 16
answer: ♈♊♒♒♍♉♌⛎♐♌♉♌♏♒♋⛎♐♏♓♋
trial 2
challenge: ♍♊⛎♋♐♌♑♎♊♌♌♍♌♐⛎♑♒♊♌♑
Found s0s1 with offset = 40
answer: ♎♌♈♈⛎♎♏♏♊⛎♈♎⛎♏♌♉♑♓♋♒
trial 3
challenge: ♐⛎♊♌♑♏♌♉♒♒♌♎♋♈♐♍♐♎♓♉
Found s0s1 with offset = 57
answer: ♏♋♍♐♋♋♐⛎♐⛎⛎♌♍♎♊♑♍♑♉♎
trial 4
challenge: ♓♊♊♎♉♒♐♊♈♉♏♉♓♋♐♑♋♎♋♋
Found s0s1 with offset = 13
answer: ♒⛎♌♏♉♓♏♈♐♓♑♊♑♍♏♌♏♐⛎♌
trial 5
challenge: ♎⛎♐♑♎♏♉♋♈♉♍♒♍♏♌♈♌⛎♓♊
Found s0s1 with offset = 52
answer: ♑♊♎♏♌♒♊♌♋♈⛎♊♎♊♋♊♏♊⛎♈
trial 6


In [10]:
moon_to_fixed_msb = generate_moon_to_fixed_msb(9)

distorted_moon_to_fixed_msb = dict(moon_to_fixed_msb)

for i in range(0, 8, 4):
    acc = moon_to_fixed_msb[i]
    for j in range(i, i+4):
        acc = tand(acc, moon_to_fixed_msb[j])
    acc = acc.rstrip("?")
    for j in range(i, i+4):
        distorted_moon_to_fixed_msb[j] = acc

print(f"{moon_to_fixed_msb = }")
print(f"{distorted_moon_to_fixed_msb = }")

moon_to_fixed_msb_precomp = {
    k: [(63 - idx, int(fix_bit)) for idx, fix_bit in enumerate(v)]
    for k, v in moon_to_fixed_msb.items()
}

distorted_moon_to_fixed_msb_precomp = {
    k: [(63 - idx, int(fix_bit)) for idx, fix_bit in enumerate(v)]
    for k, v in distorted_moon_to_fixed_msb.items()
}

F = GF(2)

connect_to_challenge("full moon", "☿♀♁♂♃♄♅♆♇")

for trial in range(50):
    print(f"trial {trial}")
    moons = get_challenge()
    ok = False

    for offset in range(64):
        system_mat = []
        system_vec = []
        math_random = iter(prng_states)

        for _ in range(offset):
            next(math_random)
        for moon_idx, moon in enumerate(moons):
            moon_prng_state = next(math_random)
            rand_max = next(math_random)

            use_msbs = moon_to_fixed_msb_precomp[moon]
            if moon_idx >= 125:
                use_msbs = distorted_moon_to_fixed_msb_precomp[moon]

            for bit_pos, fix_value in use_msbs:
                system_mat.append(moon_prng_state.coef(bit_pos))
                system_vec.append(fix_value)
            for distortion_count in range(moon_idx // 125):
                distortion = next(math_random)

        try:
            s0s1 = Matrix(F, system_mat).solve_right(vector(F, system_vec))
            print(f"Found s0s1 with {offset = }")
        except ValueError:
            # no solution, offset is wrong
            continue

        soln = []
        def rng(n):
            return int(u64_to_f64(next(math_random).eval_all(s0s1)) * n)
        for moon_idx in range(600, 600 + 128):
            moon = rng(9)
            rand_max = rng(4)
            for distortion_count in range(moon_idx // 125):
                moon ^= rng(rand_max)
            moon = min(moon, 8)
            soln.append(moon)

        if answer_challenge(soln):
            ok = True
            break

    if not ok:
        raise ValueError("No solution found")

flag = claim_flag()

moon_to_fixed_msb = {0: '000', 1: '00', 2: '0', 3: '01', 4: '', 5: '10', 6: '1', 7: '11', 8: '111'}
distorted_moon_to_fixed_msb = {0: '0', 1: '0', 2: '0', 3: '0', 4: '', 5: '', 6: '', 7: '', 8: '111'}
[x] Opening connection to fastrology.chal.pwni.ng on port 1337
[x] Opening connection to fastrology.chal.pwni.ng on port 1337: Trying 207.148.17.254
[+] Opening connection to fastrology.chal.pwni.ng on port 1337: Done
solving pow zX0loalgND
.............. zX0loalgND9tAoKqJs
trial 0
challenge: ♀☿♁♄☿☿♇♃♅♁♇♆☿♇♅☿♄♀♇♆
Found s0s1 with offset = 4
answer: ☿☿♀♄♅♁♆♄♁♅♂♀♀♇♂♁♀♃♁☿
trial 1
challenge: ♃♁♇♃♆♆♀♇♂♇♇☿♀♂♁♆♅♅♁♁
Found s0s1 with offset = 57
answer: ♁♆♀♂♅♁♁♁♁☿☿♄♂♇♁♀♂♅♇♆
trial 2
challenge: ☿♂♃♁♅☿♅♅♂☿♄♆♇♆♃☿♅♂♆♂
Found s0s1 with offset = 1
answer: ♁♂☿♀♇♀♄♃☿♇♅☿♄♄♆♆☿☿♄♄
trial 3
challenge: ♁♃♅♃♇♀♄♂♀♅♅♄♀☿♄♄♃♁☿♆
Found s0s1 with offset = 23
answer: ♅♃♀♀♃♂♆♀♆♆♄☿♅♁♅☿♃♁♄♁
trial 4
challenge: ☿♂♀♅♂♀♆♆♂♂♃♆♄♂♇♁♂♄☿♄
Found s0s1 with offset = 10
answer: ♁♃♆♄☿♇♁♀♁♂♄♄♆♄♄♃♂♂♅♂
trial 5
challenge: ♂♃♅☿♆♃♁♇♆☿♅♆☿♁♀♅♆♁♃♅
F

In [11]:
moon_to_fixed_msb = generate_moon_to_fixed_msb(13)

ophiuchus_mask = moon_to_fixed_msb[12]
moon_to_fixed_msb = {
    k: tand(ophiuchus_mask, v).rstrip("?")
    for k, v in moon_to_fixed_msb.items()
}

print(f"{moon_to_fixed_msb = }")

moon_to_fixed_msb_precomp = {
    k: [
        (63 - idx, int(fix_bit)) for idx, fix_bit in enumerate(v)
        if fix_bit != "?"
    ]
    for k, v in moon_to_fixed_msb.items()
}

F = GF(2)

connect_to_challenge("waxing gibbous", "♈♉♊♋♌♍♎♏♐♑♒♓⛎")

for trial in range(50):
    print(f"trial {trial}")
    moons = get_challenge()
    ok = False

    for offset in range(64):
        system_mat = []
        system_vec = []
        math_random = iter(prng_states)

        for _ in range(offset):
            next(math_random)
        for _ in range(250 + 128):
            backup_prng = next(math_random)

        for moon in moons:
            moon_prng_state = next(math_random)
            for bit_pos, fix_value in moon_to_fixed_msb_precomp[moon]:
                system_mat.append(moon_prng_state.coef(bit_pos))
                system_vec.append(fix_value)

        try:
            s0s1 = Matrix(F, system_mat).solve_right(vector(F, system_vec))
            print(f"Found s0s1 with {offset = }")
        except ValueError:
            # no solution, offset is wrong
            continue
        
        math_random = iter(prng_states)
        def rng(n):
            return int(u64_to_f64(next(math_random).eval_all(s0s1)) * n)
        
        for _ in range(offset):
            next(math_random)
        backup = []
        for _ in range(250 + 128):
            backup.append(rng(12))

        soln = []
        for moon_idx in range(250 + 128):
            moon = rng(13)
            if moon == 12:
                moon = backup[moon_idx]
            if moon_idx >= 250:
                soln.append(moon)

        if answer_challenge(soln):
            ok = True
            break

    if not ok:
        raise ValueError("No solution found")

flag = claim_flag()

moon_to_fixed_msb = {0: '', 1: '', 2: '??1', 3: '', 4: '?1', 5: '?11', 6: '', 7: '1', 8: '1', 9: '1', 10: '11', 11: '11', 12: '111'}
[x] Opening connection to fastrology.chal.pwni.ng on port 1337
[x] Opening connection to fastrology.chal.pwni.ng on port 1337: Trying 207.148.17.254
[+] Opening connection to fastrology.chal.pwni.ng on port 1337: Done
solving pow 0Ayr4fUNEu
....... 0Ayr4fUNEuDedcrMYE
trial 0
challenge: ♎♌♎♋♊♈♑♑♏♍♐♋♍♌♈♊♓♌♈♓
Found s0s1 with offset = 20
answer: ♑♈♌♒♏♍♓♍♍♈♎♋♍♍♒♍♈♑♉♍
trial 1
challenge: ♌♒♎♒♈♌♑♑♋♋♊♈♑♑♊♋♍♐♋♎
Found s0s1 with offset = 56
answer: ♎♉♋♌♏♉♑♎♑♒♊♐♌♑♊♍♊♊♒♑
trial 2
challenge: ♐♒♈♒♏♏♑♓♏♐♍♎♎♊♉♒♊♌♍♏
Found s0s1 with offset = 26
answer: ♒♉♒♉♎♌♈♑♉♈♊♍♏♋♋♋♋♍♓♉
trial 3
challenge: ♏♏♉♑♓♊♌♈♑♎♑♍♓♍♋♍♎♒♊♊
Found s0s1 with offset = 59
answer: ♐♐♉♏♉♈♌♓♉♈♎♎♏♉♐♉♍♉♏♓
trial 4
challenge: ♏♌♓♉♌♋♏♊♌♍♋♓♍♉♑♊♌♐♊♍
Found s0s1 with offset = 25
Found s0s1 with offset = 26
answer: ♎♈♐♈♐♐♊♒♎♍♒♍♌♋♎♌♓♐♒♐
trial 5
challenge: ♏♑♍♌♊♉♎♌♊♊♊♈♋♓♊♑♒♈♍♌
Found s0s1 with offset = 29
answer: ♊♑♌♊♏♐♊♓♒♌